En esta libreta utilizamos dos librerias que nos permiten comunicarnos con la API de Google Maps, la primera googlemaps será usada para conseguir las distancias en km entre ciudades, la segunda, gmaps será usada para mostrar gráficamente los mapas, marcadores y rutas.

Esta libreta se utilizó independientemente debido a que para llenar la matriz de distancias hay que hacer alrededor de 8000 llamadas a la API de Google Maps y debido a que Google Maps tiene en sus términos y condiciones que está prohibido hacer descargas grandes de información se optó mejor por usarla solo una vez, guardar la información necesaria en archivos CSV y trabajar con estos desde otra libreta de Jupyter.

Cabe mencionar que llenar esta matriz de distancias provocó que mi servicio de llamadas a la API de Google Maps fuera suspendido por 24 horas.

En caso de usar anaconda los comandos para instalar las extensiones son:

conda install -c conda-forge gmaps 

conda install -c conda-forge googlemaps

In [12]:
#pip install -U googlemaps # En caso de no tenerla instalada

Note: you may need to restart the kernel to use updated packages.


In [30]:
#!pip install -U gmaps  # Descomentar en caso de tener que instalarla

In [1]:
# Librerías usadas
import numpy as np
import pandas as pd
import googlemaps
import gmaps as gm

# Carga de Datos

Iniciamos cargando la base de datos Limpia, la cual parte de la base de datos actualizada hasta 2017 pero también se le han agregado manualmente los Pueblos Mágicos con nombramientos desde 2017 a la fecha

In [4]:
urlbd = 'https://raw.githubusercontent.com/ArtemioPadilla/MxMagicTowns-PueblosMagicosMx/main/BDsPueblosMagicos/BDLimpia-PueblosMagicos.csv'
magic = pd.read_csv(urlbd, encoding='utf-8')

In [5]:
#pd.set_option('display.max_rows', 5)

In [6]:
magic # Mostramos la base de datos limpia

,estado,municipio,ano_ingreso
0,Aguascalientes,Real de Asientos,2006
1,Aguascalientes,Calvillo,2012
2,Aguascalientes,San Jose de Gracia,2015
3,Baja California,Tecate,2012
4,Baja California Sur,Todos Santos,2006
...,...,...,...
127,Zacatecas,Teul de Gonzalez Ortega,2011
128,Zacatecas,Sombrerete,2012
129,Zacatecas,Pinos,2012
130,Zacatecas,Nochistlan,2012


# Coordenadas

En esta sección se agregan las coordenadas de cada Pueblo Mágico haciendo llamadas a la API de Google Maps Geocode

Agregamos a nuestro Dataframe las columnas de Latitud y Longitud

In [7]:
magic['latitud']=''
magic['longitud']=''

Importamos la API de Google.

Se necesita una llave válida para acceder a la API de Google Maps

In [8]:
# Inicialiamos la API con nuestra llave
api_key = "KEY" # Inserte su llave
gmaps = googlemaps.Client(key=api_key)

Hacemos una prueba de formato para obtener las coordenadas de una ubicación de ejemplo

In [9]:
# Probamos con Puerto Vallarta
loc = gmaps.geocode('Puerto Vallarta, Jalisco')
# Vemos la latitud y la longitud
loc[0]['geometry']['location']['lat'], loc[0]['geometry']['location']['lng']

(20.653407, -105.2253316)

Llenamos las coordenadas de nuestra base de datos

In [8]:
for idx in magic.index:
  #print(magic['municipio'][idx],magic['estado'][idx])
  loc = gmaps.geocode(magic['municipio'][idx]+','+magic['estado'][idx])
  #print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)
  magic['longitud'].iloc[idx]=loc[0]['geometry']['location']['lng']
  magic['latitud'].iloc[idx]=loc[0]['geometry']['location']['lat']

Verificamos la operación para el estado de Sinaloa

In [9]:
magic[magic.estado=='Sinaloa']

,estado,municipio,ano_ingreso,latitud,longitud
105,Sinaloa,Cosala,2005,24.516165,-106.796633
106,Sinaloa,El Fuerte,2009,26.236505,-108.660443
107,Sinaloa,El Rosario,2012,23.089563,-105.678816
108,Sinaloa,Mocorito,2015,25.363947,-107.818938


Utilizamos una precisión al mostrar de 10 decimales dado que las coordenadas utilizan muchos decimales significativos

In [10]:
pd.set_option("display.precision", 10)

Con estos cambios mostramos nuestro nuevo Dataframe

In [11]:
magic

,estado,municipio,ano_ingreso,latitud,longitud
0,Aguascalientes,Real de Asientos,2006,22.2381386,-102.0897755
1,Aguascalientes,Calvillo,2012,21.84962425,-102.7125427323
2,Aguascalientes,San Jose de Gracia,2015,22.1426267,-102.5301357325
3,Baja California,Tecate,2012,32.5653831,-116.6298787
4,Baja California Sur,Todos Santos,2006,23.450067,-110.225374
...,...,...,...,...,...
127,Zacatecas,Teul de Gonzalez Ortega,2011,21.3503371,-103.4050259549
128,Zacatecas,Sombrerete,2012,23.57451835,-103.6468284589
129,Zacatecas,Pinos,2012,22.2729743,-101.6040748007
130,Zacatecas,Nochistlan,2012,22.7850878,-102.5735723


# Matriz de Distancias

In [15]:
# Configuramos gmaps con nuestra api_key
gm.configure(api_key=api_key)

Haremos un mapa con todas las ubicaciones de los Pueblos Mágicos, para esto crearemos una lista con todas las coordenadas y luego la graficaremos con gmaps

In [16]:
marker_locations = [] # Lista para guardar las coordenadas
for p_magico in magic.values:
    marker_locations.append(p_magico[-2:])  # Guardamos las coordenadas de cada Pueblo Mágico

In [17]:
fig = gm.figure() # Creamos la figura del mapa
markers = gm.symbol_layer(marker_locations, fill_color='red', stroke_color='red', scale=2) # Definimos los marcadores y sus propiedades
fig.add_layer(markers) # Agregamos los marcadores a la figura
fig  # Mostramos la figura

Figure(layout=FigureLayout(height='420px'))

Ahora procedemos a crear una matriz de distancias entre Pueblos Mágicos

In [23]:
# Inicializamos una matriz cuadrada del tamaño de la cantidad de pueblos mágicos que tenemos en ceros
dist = np.zeros((magic.shape[0],magic.shape[0]),dtype=float) 

In [60]:
# La mostramos
dist, dist.shape

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 (132, 132))

In [20]:
magic.iloc[0]

estado           Aguascalientes
municipio      Real de Asientos
ano_ingreso                2006
latitud              22.2381386
longitud           -102.0897755
Name: 0, dtype: object

Probamos el uso de matrices de distancia con la API de Google Maps

In [39]:
dist_cdmx_vallarta = gmaps.distance_matrix("Ciudad de México", "Puerto Vallarta, Jalisco", mode="driving")
dist_cdmx_vallarta

{'destination_addresses': ['Puerto Vallarta, Jalisco, Mexico'],
 'origin_addresses': ['Mexico City, CDMX, Mexico'],
 'rows': [{'elements': [{'distance': {'text': '879 km', 'value': 878868},
     'duration': {'text': '10 hours 41 mins', 'value': 38446},
     'status': 'OK'}]}],
 'status': 'OK'}

Usaremos la distancia en KM para este proyecto.

Hacemos una prueba para ver como acceder a este dato

In [48]:
dist_cdmx_vallarta['rows'][0]['elements'][0]['distance']['value']/1000  # Accedemos a la distancia en metros, dividimos entre 1000 para obtenerla en km

878.868

Sacaremos las distancias entre pueblos mágicos, asumiremos simetría en las distancias.

In [61]:
for idx in magic.index: # Para cada pueblo mágico
  for idx2 in magic.index: # Lo comparamos con cada otro pueblo mágico
    if idx <= idx2: # Solo calculamos de ida, suponemos simetria para ahorrar requests a la API de Google
      p_magico1 = str(magic.iloc[idx].municipio+', '+magic.iloc[idx].estado) # Primer pueblo mágico
      p_magico2 = str(magic.iloc[idx2].municipio+', '+magic.iloc[idx2].estado) # Segundo Pueblo Mágico
      # Calculamos la distancia
      dist_ = gmaps.distance_matrix(p_magico1, p_magico2, mode="driving")['rows'][0]['elements'][0]['distance']['value']/1000 
      dist[idx][idx2]=dist_ # Guardamos las entradas de ida
      dist[idx2][idx]=dist_ # Guardamos las entradas de regreso

Con esto obtenemos la matriz de distancias entre cada pueblo mágico

In [64]:
dist

array([[  0.   , 112.507,  48.605, ...,  70.134, 167.172,  97.503],
       [112.507,   0.   ,  95.722, ..., 187.772,  90.259, 163.22 ],
       [ 48.605,  95.722,   0.   , ..., 116.852, 152.639,  95.062],
       ...,
       [ 70.134, 187.772, 116.852, ...,   0.   , 225.732, 138.337],
       [167.172,  90.259, 152.639, ..., 225.732,   0.   , 217.417],
       [ 97.503, 163.22 ,  95.062, ..., 138.337, 217.417,   0.   ]])

La distancia máxima entre pueblos mágicos es

In [63]:
np.max(dist)

4343.399

Guardamos la matriz obtenida para no tenerla que volver a calcular dado que usa alrededor de 8000+ requests a la API de google

In [67]:
np.savetxt("dist_matrix.csv", dist, delimiter=",")

Creamos un Dataframe para poder poner nombres a las columnas y a los renglones

In [69]:
dist_df = pd.DataFrame(dist) # Creamos DF

In [72]:
cols = [] # Guadaremos los nombres en cols
for idx in magic.index: # Recorremos todos los pueblos mágicos
    cols.append(str(magic.iloc[idx].estado+ ', '+ magic.iloc[idx].municipio))  # Guardamos su nombre empezando con el estado
dist_df.columns= cols # Colocamos los nombres de las columnas
dist_df.index=cols # Colocamos los nombres de los renglones
dist_df # Mostramos el Dataframe

,"Aguascalientes, Real de Asientos","Aguascalientes, Calvillo","Aguascalientes, San Jose de Gracia","Baja California, Tecate","Baja California Sur, Todos Santos","Baja California Sur, Loreto","Campeche, Isla Aguada","Campeche, Palizada","Coahuila, Parras de la Fuente","Coahuila, Cuatro Cienegas",...,"Yucatan, Izamal","Yucatan, Valladolid","Yucatan, Mani","Yucatan, Sisal","Zacatecas, Jerez de Garcia Salinas","Zacatecas, Teul de Gonzalez Ortega","Zacatecas, Sombrerete","Zacatecas, Pinos","Zacatecas, Nochistlan","Zacatecas, Guadalupe"
"Aguascalientes, Real de Asientos",0.000,112.507,48.605,2243.690,1377.484,1646.573,1509.007,1473.996,601.257,738.511,...,1914.125,2005.263,1855.313,1877.977,157.089,264.714,264.624,70.134,167.172,97.503
"Aguascalientes, Calvillo",112.507,0.000,95.722,2309.407,1404.445,1673.534,1493.304,1458.293,666.974,804.228,...,1898.421,1989.560,1839.609,1862.274,159.021,151.023,330.341,187.772,90.259,163.220
"Aguascalientes, San Jose de Gracia",48.605,95.722,0.000,2241.248,1375.042,1644.132,1494.475,1459.463,598.815,736.069,...,1899.592,1990.730,1840.780,1863.444,154.648,247.657,262.182,116.852,152.639,95.062
"Baja California, Tecate",2243.690,2309.407,2241.248,0.000,1560.577,1128.928,3682.107,3647.096,1943.174,2149.735,...,4087.224,4178.363,4028.412,4051.077,2150.745,2291.301,2043.825,2287.103,2342.710,2153.719
"Baja California Sur, Todos Santos",1377.484,1404.445,1375.042,1560.577,0.000,433.282,2712.899,2677.887,1415.456,1486.815,...,3118.016,3209.154,3059.204,3081.868,1305.177,1322.093,1136.360,1441.535,1373.501,1308.151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Zacatecas, Teul de Gonzalez Ortega",264.714,151.023,247.657,2291.301,1322.093,1599.983,1630.990,1596.760,711.722,849.192,...,2036.332,2128.862,1978.863,1999.364,159.771,0.000,329.387,356.988,173.984,221.302
"Zacatecas, Sombrerete",264.624,330.341,262.182,2043.825,1136.360,1414.249,1720.422,1686.192,454.331,528.536,...,2125.763,2218.293,2068.295,2088.795,171.220,329.387,0.000,307.545,389.018,174.162
"Zacatecas, Pinos",70.134,187.772,116.852,2287.103,1441.535,1719.424,1461.821,1427.591,646.511,783.982,...,1867.162,1959.692,1809.693,1830.194,197.537,356.988,307.545,0.000,225.732,138.337
"Zacatecas, Nochistlan",167.172,90.259,152.639,2342.710,1373.501,1651.391,1501.876,1467.646,727.984,865.454,...,1907.218,1999.747,1849.749,1870.250,222.028,173.984,389.018,225.732,0.000,217.417


In [73]:
dist_df.to_csv('dist_df.csv') # Guardamos el Dataframe con la matriz de distancias y los nombres

Guardamos también nuestro Dataframe con Coordenadas

In [77]:
magic.to_csv("pueblos_magicos_coordenadas.csv")

Con los datos guardados en CSV cambiaremos de Libreta para no correr el riesgo de volver a hacer las request a la API de Google. Llenar la matriz de distancias asumiendo simetría cuesta 132*132/2=8712 requests, cada 1000 requests cuestan 5 dolares, por lo que llenar esta matriz de distancias costaría alrededor de $43 dolares. 